In [2]:
from vmad import Builder
from vmad import autooperator
from vmad import operator
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d 
from scipy.integrate import quad
from scipy.misc import derivative
from nbodykit.cosmology import Planck15

import matplotlib.pyplot as plt

In [2]:
%matplotlib notebook

In [32]:
cosmo.get_pk()

0.25813909378966493

In [3]:
cosmo = Planck15.match(Omega0_m=.308)
cosmo = cosmo.match(sigma8=.8158)

To find the comoving distance as a function of reshift we use the equation from Peebles 1993

\begin{equation}
\chi = \dfrac{c}{H_0}\int_0^z\dfrac{dz'}{E(z')^{1/2}}
\end{equation}

Where 

\begin{equation}
E(\Omega_{m0}, \Omega_{r0}, \Omega_{\Lambda 0}, z ) = \Omega_{m0}(1+z)^3+\Omega_{r0}(1+z)^2+\Omega_{\Lambda 0}
\end{equation}.

Using ```scipy.integrate.quad```, We can integrate this to find the redshifts to different redshifts. The jacobian of the comoving distance with respect to $\Omega_{m0}, \Omega_{r0}, \Omega_{\Lambda 0}, z$ is , 

\begin{equation}
\textbf{J} = 
\begin{split}
    \begin{bmatrix}
    \partial_{\Omega_{m0}} \chi(\Omega_{m0}, \Omega_{r0}, \Omega_{\Lambda 0}, z)\\
    \partial_{\Omega_{r0}} \chi(\Omega_{m0}, \Omega_{r0}, \Omega_{\Lambda 0}, z)\\
    \partial_{\Omega_{\Lambda 0}} \chi(\Omega_{m0}, \Omega_{r0}, \Omega_{\Lambda 0}, z)\\
    \partial_{z}\chi(\Omega_{m0}, \Omega_{r0}, \Omega_{\Lambda 0}, z)
    \end{bmatrix}
\end{split}
\end{equation}

Which becomes, 
\begin{equation}
\textbf{J} = 
\begin{split}
\begin{bmatrix}
\dfrac{-c}{2H_0}\int_0^z \dfrac{ (1+z')^3}{E(\Omega_{m0}, \Omega_{r0}, \Omega_{\Lambda 0}, z')^{3/2}}dz'\\
\dfrac{-c}{2H_0} \int_0^z \dfrac{ (1+z')^2}{E(\Omega_{m0}, \Omega_{r0}, \Omega_{\Lambda 0}, z')^{3/2}}dz'\\
\dfrac{-c}{2H_0} \int_0^z \dfrac{ 1}{E(\Omega_{m0}, \Omega_{r0}, \Omega_{\Lambda 0}, z')^{3/2}}dz'\\
\dfrac{c}{H_0}\dfrac{1}{E(\Omega_{m0}, \Omega_{r0}, \Omega_{\Lambda 0}, z')^{1/2}}
\end{bmatrix}
\end{split}
\end{equation}

So that if theres a vector which is, 
\begin{equation}
\vec{v} = 
\begin{bmatrix}
v_1,  \dots, v_4
\end{bmatrix}
\end{equation}
which has dimensions of the output paramater.
The vjp is defined as, 
\begin{equation}
vjp = \vec{v}\textbf{J}
\end{equation}
which has dimensions of the input paramaters. Similarly, if the vector is, 
\begin{equation}
\vec{v} = 
\begin{bmatrix}
v_1
\end{bmatrix}
\end{equation}
which has dimensions of the input parameters and the jvp is defined as 
\begin{equation}
jvp = \textbf{J}\vec{v}
\end{equation}
Which has size of the input parameters

Because of the flat space time, angular diameter distance is equal to comoving distance and we let $\Omega_k\sim\Omega_r\sim 0$ so that $\Omega_\Lambda = 1-\Omega_m$. This lets us simplify the equation to let, 
\begin{equation}
E(\Omega_{m0},z') = \Omega_{m0}((1+z)^3 -1)+1
\end{equation}
and the Jacobian simplifies to two terms, 
\begin{equation}
\textbf{J} = 
\begin{split}
\begin{bmatrix}
\dfrac{-c}{2H_0}\int_0^z \dfrac{(1+z')^3-1}{E(\Omega_{m0}, z')^{3/2}}dz'\\
\dfrac{c}{H_0}\dfrac{1}{E(\Omega_{m0}, z')^{1/2}}
\end{bmatrix}
\end{split}
\end{equation}

In [15]:
#Vanessas operator 
@operator
class z_chi:
    """
    go from redshift distance to comoving
    """
    ain  = {'z' : 'ndarray'}
    aout = {'chi': 'ndarray'}

    def apl(node, z, cosmo):
        return dict(chi = cosmo.comoving_distance(z))
    
    def vjp(node, _chi, z, cosmo):
        res = 1./cosmo.efunc(z)/cosmo.H0*cosmo.C
        return dict(_z = np.multiply(res,_chi))
    
    def jvp(node, z_, z, cosmo):
        res = 1./cosmo.efunc(z)/cosmo.H0*cosmo.C
        return dict(chi_ = np.multiply(res,z_))

In [33]:
#Max's Operator for matter
@operator
class z_chi_matter:
    ain  = {'omega0_m': 'float', 'z':'float'}
    aout = {'chi':'float'}

    def apl(node, omega0_m, z):
        #Calculate the integral from 0->z
        E         = lambda x: (omega0_m  * ((1+x)**3 -1)+1)**(-1/2)
        Dc , _    = quad(E, 0, z)
        return dict(chi = Dc*cosmo.C/cosmo.H0)
    
    def vjp(node, _chi, omega0_m, z):
        
        #Return the derivative of the integral WR2 omega0_m and mult by _chi
        _omega0_m  =  _chi  *  quad(deriv_integral, 0, z, args=(omega0_m))[0] 
        
        
        #Calculate the derivative of the integral with respect to z i.e. the original function
        _z         =  _chi  * (omega0_m * ((1+z)**3 -1)+1)**(-1/2) 
        
        #Multiply by hubble distance and return
        return dict(_omega0_m = _omega0_m*cosmo.C/cosmo.H0, _z= _z*cosmo.C/cosmo.H0 )
    
    def jvp(node, omega0_m_, z_, omega0_m, z):
        
        #Find derivative with respevct to omega_0 and mult by omega0_m_
        omega0_m_   *= omega0_m_ * quad(deriv_integral, 0, z, args=(omega0_m))[0]
        
        #Do same for z
        z_          *=  (omega0_m * ((1+z)**3 -1) +1)**(-1/2)
        
        #Multiply by hubble distance
        return dict(chi_ = omega0_m_*cosmo.C/cosmo.H0   + z_*cosmo.C/cosmo.H0 )
    


In [34]:
def deriv_integral(x, omega0_m):
    """
    Derivative of the comoving distance with respect to matter density
    
    """
    
    #Create the denominator of the integral
    E = (omega0_m * ((1+x)**3 -1)+ 1)**(-3/2)
    
    diriv_factor = (-1/2) * ((1+x)**3-1)

    return E*diriv_factor

def finite_difference(x, h):
    """
    Forward differencing model for matter density
    """
    om0 = cosmo.Omega0_m
    denom_1 = ((om0+h)*((1+x)**3-1)+1)**(-1/2)
    denom_2 = (om0*((1+x)**3-1)+1)**(-1/2)
    const = cosmo.C/(h*cosmo.H0)
    
    
    return const*(denom_1-denom_2)

In [35]:
@autooperator('z->chi')
def z_chi_van(z):
    f =  z_chi(z, cosmo)
    return dict(chi=f)

@autooperator('m0, z->chi')
def z_chi_matter_1(m0, z):
    f= z_chi_matter(m0, z)
    return dict(chi=f)

In [36]:
#Compute the model for vanessas and my operators and check errors are small
model_van = z_chi_van.build()
chi_van,tape_van = model_van.compute(init=dict(z=1), 
                                     vout='chi',
                                     return_tape=True)



model_matter = z_chi_matter_1.build()
chi_matter,tape_matter = model_matter.compute(init=dict(m0 =cosmo.Omega0_m , z=1), 
                                              vout='chi', 
                                              return_tape=True)


error = np.abs(np.asarray(chi_matter)- np.asarray(chi_van))/np.asarray(chi_van) < 1e-4
print('chi error less than .01%: ', error)


chi error less than .01%:  True


In [37]:
#Compute the vjp for vanessas and my model and check that errors are small, 
#Use the finite difference function for matter density to make sure errors are small
vjp_init = dict(_chi = 1)


vjp_matter = tape_matter.get_vjp()
_m0, _z_matter= vjp_matter.compute(init=vjp_init, vout=['_m0', '_z'])


vjp_vanessa = tape_van.get_vjp()
_z= vjp_vanessa.compute(init=vjp_init, vout=['_z'])

#Check that error is small
error_z = np.abs(_z[0]- _z_matter)/_z < 1e-3
error_m = np.abs(quad(finite_difference, 0, 1, args = (.00000001))[0] - _m0)/quad(finite_difference, 0, 1, args = (.00000001))[0] < 1e-5

print('_z error less than .1%: ', error_z[0])
print('_m error less than .001%: ', error_m)

_z error less than .1%:  True
_m error less than .001%:  True


In [38]:
#Check the JVP
jvp_init = dict(m0_ = 0, z_ = 1)

jvp_matter = tape_matter.get_jvp()
chi_matter_ = jvp_matter.compute(init=jvp_init, vout=['chi_'])

jvp_init = dict(z_ = 1)
jvp_vanessa = tape_van.get_jvp()
chi_van_ = jvp_vanessa.compute(init=jvp_init, vout=['chi_'])

error = np.abs(chi_van_[0]-chi_matter_)/chi_van_[0] <1e-3
print('chi_ error less than .1%', error[0])

chi_ error less than .1% True


In [43]:
chi_matter

2301.8042810349148

# Current matter density and maximum distance


In [12]:
#Current omega0_m 
Omega0_m = .3153
error    = .0073
Omega0_m_plus4x = Omega0_m+4*error
Omega0_m_minus4x = Omega0_m-4*error

cosmo_plus4x  = Planck15.match(Omega0_m=Omega0_m_plus4x)
cosmo_plus4x = cosmo_plus4x.match(sigma8=.8158)
cosmo_minus4x = Planck15.match(Omega0_m=Omega0_m_minus4x)
cosmo_minus4x = cosmo_minus4x.match(sigma8=.8158)

chi_plus4x = cosmo_plus4x.comoving_distance(2)
chi_minus4x = cosmo_minus4x.comoving_distance(2)

chi_plus4x, chi_minus4x

(3496.212085878297, 3669.5871332814636)

$\Omega_{0,m} +4\sigma = 3496.212085878297 \ \ \ \ \ \ $
$\Omega_{0,m} -4\sigma = 3669.5871332814636$

3496.212085878297 3669.5871332814636


In [7]:
from vmad.core import stdlib

In [8]:
def test(a):
    return 23*a

In [16]:
@autooperator('a, b->c')
def trying(a,b):
    stdlib.assert_true(a, lambda a: a>0)
    print(x)
    return dict(c=x)

In [17]:
m = trying.build()

NameError: name 'x' is not defined

In [18]:
m.compute(init = dict(a=1, b=0), vout='c')

[]